In [1]:
using ProgressBars
using ITensors
using PastaQ
using RandomMeas

[ Info: Precompiling PastaQ [30b07047-aa8b-4c78-a4e8-24d720215c19]


In [2]:
N = 2
depth = 2
circuit    = randomcircuit(N, depth=depth)
noisemodel = (1 => ("depolarizing", (p = 0.02,)),2 => ("depolarizing", (p = 0.02,)))
noisemodel0 = (1 => ("depolarizing", (p = 0.0,)),2 => ("depolarizing", (p = 0.0,)))
Λ0 = runcircuit(circuit; process = true, noise = noisemodel0)
ξ0 = [firstind(Λ0[i],tags="Input",plev=0) for i in 1:N]
ξ20 = [firstind(Λ0[i],tags="Output",plev=0) for i in 1:N]
Λ = runcircuit(circuit; process = true, noise = noisemodel)
ξ = [firstind(Λ[i],tags="Input",plev=0) for i in 1:N]
ξ2 = [firstind(Λ[i],tags="Output",plev=0) for i in 1:N]
for i in 1:N
    Λ0[i] = replaceind(Λ0[i],ξ0[i],ξ[i])
    Λ0[i] = replaceind(Λ0[i],ξ0'[i],ξ'[i])
    Λ0[i] = replaceind(Λ0[i],ξ20[i],ξ2[i])
    Λ0[i] = replaceind(Λ0[i],ξ20'[i],ξ2'[i])
end
F = fidelity(Λ,Λ0)
println(" -- Process Fidelity ", F)

 -- Process Fidelity 0.9604266666666721


In [3]:
nu = 4000
NM = 10000
states = ["Dn" for n in 1:N]
ψ  = MPS(ComplexF64,ξ,states)
ρ = outer(ψ',ψ)
data = zeros(Int8,(NM,N))
shadow = ITensor(vcat(ξ,ξ'))
Λs = ITensor(vcat(ξ,ξ',ξ2,ξ2'))
Λs_CRM = ITensor(vcat(ξ,ξ',ξ2,ξ2'))

P0 = state(ξ[1],"Dn")
for i in 2:N
    P0 *= state(ξ[i],"Dn")
end

Fs = zeros(Float64,nu)
Fs_CRM = zeros(Float64,nu)

for r in ProgressBar(1:nu, printing_delay=2)   
        ui = get_rotations(ξ,1) #Haar rotations in A
        uf = get_rotations(ξ,1) #Haar rotations in A
        ### Quantum Measurements
        ρu = apply(ui,ρ,apply_dag=true)
        ρe = runcircuit(ρu,circuit,noise=noisemodel)
        #Build shadow for the initial state
        uid = [dag(swapinds(ui[i],ξ[i],ξ[i]')) for i in 1:N]
        ρi = get_shadow(P0,ξ,uid)
        ρi = swapinds(ρi,ξ,ξ')

        #Build shadow for the final state
        data = get_RandomMeas(ρe,uf,NM)
        P = get_Born(data,ξ)
        ρf = get_shadow(P,ξ,uf)
        replaceinds!(ρf,ξ,ξ2)
        replaceinds!(ρf,ξ',ξ2')

        #Form process shadow
        Λt = 2^N*ρi*ρf
        Λs += Λt
        Fs[r] = real(inner(Λs,flatten(Λ0))/2^(2*N))/r
    
        #Repeat the same thing for simulated measurements
        σe = runcircuit(ρu,circuit,noise=noisemodel0)
        Pσ = get_Born(apply(uf,σe,apply_dag=true))
        σf = get_shadow(Pσ,ξ,uf)
        replaceinds!(σf,ξ,ξ2)
        replaceinds!(σf,ξ',ξ2')
        Λt_CRM = Λt-2^N*ρi*σf+flatten(Λ0)
        Λs_CRM += Λt_CRM
        Fs_CRM[r] = real(inner(Λs_CRM,flatten(Λ0))/2^(2*N))/r
end

0.0%┣                                         ┫ 0/4.0k [00:03<-3:-4:-56, -3s/it]
0.0%┣                                         ┫ 1/4.0k [00:23<Inf:Inf, InfGs/it]
3.6%┣█▋                                          ┫ 144/4.0k [00:25<11:13, 6it/s]
7.3%┣███▏                                       ┫ 293/4.0k [00:27<05:42, 11it/s]
10.9%┣████▋                                     ┫ 436/4.0k [00:29<03:57, 15it/s]
14.2%┣██████                                    ┫ 568/4.0k [00:31<03:07, 18it/s]
17.6%┣███████▍                                  ┫ 706/4.0k [00:33<02:34, 21it/s]
21.3%┣█████████                                 ┫ 852/4.0k [00:35<02:09, 24it/s]
25.1%┣██████████▎                              ┫ 1.0k/4.0k [00:37<01:51, 27it/s]
28.9%┣███████████▉                             ┫ 1.2k/4.0k [00:39<01:36, 30it/s]
32.6%┣█████████████▍                           ┫ 1.3k/4.0k [00:41<01:25, 32it/s]
36.3%┣██████████████▉                          ┫ 1.5k/4.0k [00:43<01:16, 34it/s]
40.1%┣████████████████▍     

In [4]:
using Plots
plot(1:nu,Fs,label="estimated")
plot!(1:nu,Fs_CRM,label="CRM estimated")
plot!(1:nu,F*ones(nu),"--k",label="exact")
xscale!("log")
yscale!("log")
ylim(ymax=1)
ylabel!("Fidelity")
xlabel!("unitaries")

LoadError: Cannot convert String to series data for plotting